In [42]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/auto-mpg.csv") # mpg - расход топлива (miles per gallon)

data = data.drop(["car name", "origin"], axis=1)

data["horsepower"] = data["horsepower"].replace({"?": np.nan}).astype(np.float32)
mean_hp = np.nanmean(data["horsepower"])
data["horsepower"] = data["horsepower"].fillna(mean_hp).astype(np.uint16)

data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year
0,18.0,8,307.0,130,3504,12.0,70
1,15.0,8,350.0,165,3693,11.5,70
2,18.0,8,318.0,150,3436,11.0,70
3,16.0,8,304.0,150,3433,12.0,70
4,17.0,8,302.0,140,3449,10.5,70


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

y = data[["mpg"]].to_numpy().ravel()
X = data.drop(["mpg"], axis=1).to_numpy()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y.reshape(-1, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape

((318, 6), (80, 6))

In [47]:
from gradient_boosting import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Создаем и обучаем модель
model = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 0.11580412535921958
R2: 0.8991017744050983


In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor as SklearnGBR

# Параметры для обеих моделей
params = {
    'n_estimators': 300,
    'learning_rate': 0.01,
    'max_depth': 3,
    'subsample': 0.8,
    'random_state': 42
}

# Создаем модели
our_model = GradientBoostingRegressor(**params)
sklearn_model = SklearnGBR(**params)

# Выполняем кросс-валидацию для обеих моделей
n_folds = 5
our_mse_scores = -cross_val_score(our_model, X, y, 
                                 scoring='neg_mean_squared_error', 
                                 cv=n_folds)
our_r2_scores = cross_val_score(our_model, X, y, 
                               scoring='r2', 
                               cv=n_folds)

sklearn_mse_scores = -cross_val_score(sklearn_model, X, y, 
                                    scoring='neg_mean_squared_error', 
                                    cv=n_folds)
sklearn_r2_scores = cross_val_score(sklearn_model, X, y, 
                                  scoring='r2', 
                                  cv=n_folds)

# Выводим результаты
print("Результаты кросс-валидации (5 фолдов):\n")
print("Наша реализация:")
print(f"MSE: {our_mse_scores.mean():.4f} (+/- {our_mse_scores.std() * 2:.4f})")
print(f"R2: {our_r2_scores.mean():.4f} (+/- {our_r2_scores.std() * 2:.4f})")

print("\nSklearn реализация:")
print(f"MSE: {sklearn_mse_scores.mean():.4f} (+/- {sklearn_mse_scores.std() * 2:.4f})")
print(f"R2: {sklearn_r2_scores.mean():.4f} (+/- {sklearn_r2_scores.std() * 2:.4f})")

# Разница в процентах для средних значений
mse_diff_percent = abs(our_mse_scores.mean() - sklearn_mse_scores.mean()) / sklearn_mse_scores.mean() * 100
r2_diff_percent = abs(our_r2_scores.mean() - sklearn_r2_scores.mean()) / sklearn_r2_scores.mean() * 100

print(f"\nРазница в процентах:")
print(f"MSE: {mse_diff_percent:.2f}%")
print(f"R2: {r2_diff_percent:.2f}%")


Результаты кросс-валидации (5 фолдов):

Наша реализация:
MSE: 0.1578 (+/- 0.2406)
R2: 0.7434 (+/- 0.4057)

Sklearn реализация:
MSE: 0.1595 (+/- 0.2480)
R2: 0.7407 (+/- 0.4183)

Разница в процентах:
MSE: 1.05%
R2: 0.37%


In [49]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV


# Определяем сетку параметров для поиска
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 4],
    'subsample': [0.7, 0.8, 0.9]
}

# Создаем модель
model = GradientBoostingRegressor(random_state=42)

# Создаем объект TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Создаем объект GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,  # использовать все доступные ядра процессора
    verbose=1
)

# Выполняем поиск по сетке
grid_search.fit(X, y)

# Выводим лучшие параметры и результаты
print("\nЛучшие параметры:")
print(grid_search.best_params_)
print("\nЛучший MSE:", -grid_search.best_score_)

# Создаем DataFrame с результатами для всех комбинаций параметров
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)

# Выводим топ-5 лучших комбинаций параметров
print("\nТоп-5 лучших комбинаций параметров:")
for i in range(5):
    print(f"\n{i+1}-е место:")
    print(f"Параметры: {results.iloc[i]['params']}")
    print(f"MSE: {-results.iloc[i]['mean_test_score']:.4f} (+/- {results.iloc[i]['std_test_score']*2:.4f})")

# Создаем модель с лучшими параметрами и проверяем её на кросс-валидации
best_model = GradientBoostingRegressor(**grid_search.best_params_, random_state=42)
best_mse_scores = -cross_val_score(best_model, X, y, 
                                 scoring='neg_mean_squared_error', 
                                 cv=5)
best_r2_scores = cross_val_score(best_model, X, y, 
                               scoring='r2', 
                               cv=5)

print("\nРезультаты лучшей модели на кросс-валидации:")
print(f"MSE: {best_mse_scores.mean():.4f} (+/- {best_mse_scores.std() * 2:.4f})")
print(f"R2: {best_r2_scores.mean():.4f} (+/- {best_r2_scores.std() * 2:.4f})")

Fitting 5 folds for each of 81 candidates, totalling 405 fits

Лучшие параметры:
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100, 'subsample': 0.7}

Лучший MSE: 0.20145279985816716

Топ-5 лучших комбинаций параметров:

1-е место:
Параметры: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100, 'subsample': 0.7}
MSE: 0.2015 (+/- 0.2149)

2-е место:
Параметры: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.7}
MSE: 0.2035 (+/- 0.2001)

3-е место:
Параметры: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300, 'subsample': 0.7}
MSE: 0.2046 (+/- 0.2185)

4-е место:
Параметры: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 300, 'subsample': 0.7}
MSE: 0.2048 (+/- 0.1919)

5-е место:
Параметры: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.7}
MSE: 0.2106 (+/- 0.2334)

Результаты лучшей модели на кросс-валидации:
MSE: 0.1370 (+/- 0.1659)
R2: 0.7806 (+/- 0.2701)
